In [1]:
import random

import tqdm
import pathlib
import itertools
import collections

import os
import cv2
import numpy as np
import pandas as pd
import glob

import tensorflow as tf
from tensorflow_docs.vis import embed
from tensorflow.keras.utils import to_categorical

import imageio

from IPython import display
from urllib import request

import matplotlib.pyplot as plt


In [2]:
# Load videos 

dataset = []
video_filenames = []

for fn in glob.glob("../Videos/*"):
    video_filenames.append(fn)

In [3]:
#for filename in video_filenames:
#    with open(f"../Annotations/{filename[10:-4]}.csv", "w") as f:
#        pass


In [ ]:

# Annotationben a táblák nevei
class_names = ["timestamp", "20","30","50","60","70","80",
               "sebKorlatVege","100","120","elozniTilos",
               "teherautovalElozniTilos","keresztezodesAlarendeltUttal",
               "foutvonal","elsobbsegadas","stop","mind2BehajtaniTilos",
               "teherautovalBehajtaniTilos","behajtaniTilos",
               "veszely","veszelyesKanyarBal","veszelyesKanyarJobb",
               "veszelyesKanyarok","egyenetlenUttsest","csuszos","utszukuletJobbrol",
               "munka","fenyjelzoKeszulek","gyalogosAtkeles","gyerekek",
               "kerekparosok","?","vadVeszely","korlatozasVege",
               "kotelezoHaladasiIranyJobbra","kotelezoHaladasiIranyBalra",
               "kotelezoHaladasiIrany","kotelezoHaladasiIranyJobbraEgyenesen",
               "kotelezoHaladasiIranyBalraEgyenesen","kerulesiIranyJobbra",
               "kerulesiIranyBalra","korforgalom","elozniTilosVege",
               "teherautovalElozniTilosVege","40","autopalya","autopalyaVege",
               "autout","autoutVege","buszmegallo","egyiranyu","foutVege",
               "gyalogosAtkelo","korforgalomPiros","parkolo",
               "szembejovoForgalomnakElsobbseg","szembejovoForgalommalSzembenElsobbseg",
               "utszukulet","utszukuletBalrol","veszelyesKanyarok2","zsakutca",
               "gyalogosok","varakozniTilos","megallniTilos","jobbraKanyarodniTilos",
               "balraKanyarodniTilos","behajtaniTilosMegjeloltDolgokkal",
               "behajtaniTilosKerekparral","kerulesiIranyBalVagyJobb",
               "behajtaniTilosSulykorlat","bukkano"]


In [ ]:

# Videókhoz csv fájl rendelése
video_filenames = glob.glob("../Videos/*.mp4")

for filename in video_filenames:
    base = os.path.basename(filename)
    name = base[:-4]  # pl. 0001
    csv_path = f"../Annotations/{name}.csv"
    
    # Ha már létezik egy oszlop, azt ne írja felül
    if os.path.exists(csv_path):
        df_existing = pd.read_csv(csv_path)
        for col in class_names:
            if col not in df_existing.columns:
                df_existing[col] = 0
        df_existing.to_csv(csv_path, index=False)
    # Új oszlopot adunk hozzá
    else:
        df = pd.DataFrame(
            data=np.zeros((1000, len(class_names)), dtype=int),
            columns=class_names)
        df["timestamp"] = [x/2 for x in range(0, 1000)]
        df.to_csv(csv_path, index=False)


In [6]:

"""df = pd.DataFrame(
    data=np.zeros((1000, len(class_names)), dtype=int),
    columns=class_names)
df["timestamp"] = [x/2 for x in range(0, 1000, 1)] # TODO: set to video max frame count * x

df

#for filename in video_filenames:    
#    with open(f"../Annotations/{filename[10:-4]}.csv", "w") as f:
#        df.to_csv(f, index=False)
"""


'df = pd.DataFrame(\n    data=np.zeros((1000, len(class_names)), dtype=int),\n    columns=class_names)\ndf["timestamp"] = [x/2 for x in range(0, 1000, 1)] # TODO: set to video max frame count * x\n\ndf\n\n#for filename in video_filenames:    \n#    with open(f"../Annotations/{filename[10:-4]}.csv", "w") as f:\n#        df.to_csv(f, index=False)\n'

In [7]:

"""for filename in video_filenames:
    src = cv2.VideoCapture(str(filename))  
    print("FPS per video:", src.get(cv2.CAP_PROP_FPS ))
    print("Frame per video:",src.get(cv2.CAP_PROP_FRAME_COUNT))
    print("bitrate:",src.get(cv2.CAP_PROP_BITRATE)) 
"""

'for filename in video_filenames:\n    src = cv2.VideoCapture(str(filename))  \n    print("FPS per video:", src.get(cv2.CAP_PROP_FPS ))\n    print("Frame per video:",src.get(cv2.CAP_PROP_FRAME_COUNT))\n    print("bitrate:",src.get(cv2.CAP_PROP_BITRATE)) \n'

In [ ]:
# def format_frames(frame, output_size):

    # Pad and resize an image from a video.

    # Args:
    # frame: Image that needs to resized and padded. 
    #  output_size: Pixel size of the output frame image.

   # Return:
  #    Formatted frame with padding of specified output size.
 # """
 # frame = tf.image.convert_image_dtype(frame, tf.float32)
  #frame = tf.image.resize_with_pad(frame, *output_size)
  #return frame

def frames_from_video_file(video_path, n_frames=None, output_size = (224,224), frame_step = 15):
  """
    Creates frames from each video file present for each category.

    Args:
      video_path: File path to the video.
      n_frames: Number of frames to be created per video file.
      output_size: Pixel size of the output frame image.

    Return:
      An NumPy array of frames in the shape of (n_frames, height, width, channels).
  """
  result = []
  src = cv2.VideoCapture(str(video_path))  
  
  video_length = src.get(cv2.CAP_PROP_FRAME_COUNT)
  
  if not n_frames:
    n_frames = 1 + (video_length - 1) // frame_step

  need_length = 1 + (n_frames - 1) * frame_step

  # if need_length > video_length:
  #   start = 0
  # else:
  #   max_start = video_length - need_length
  #   start = random.randint(0, max_start + 1)
  
  start = 0

  src.set(cv2.CAP_PROP_POS_FRAMES, start)
  
  # ret is a boolean indicating whether read was successful, frame is the image itself
  ret, frame = src.read()
  result.append(format_frames(frame, output_size))

  for _ in range(n_frames - 1):
    for _ in range(frame_step):
      ret, frame = src.read()
    # ret, frame = src.read()
    if ret:
      frame = format_frames(frame, output_size)
      result.append(frame)
    else:
      result.append(np.zeros_like(result[0]))
      
  src.release()
  
  result = np.array(result)[..., [2, 1, 0]]

  return result

In [9]:
#sample_video = frames_from_video_file(video_filenames[5], n_frames = 100)
#sample_video.shape

In [10]:
"""def to_gif(images):
  converted_images = np.clip(images * 255, 0, 255).astype(np.uint8)
  imageio.mimsave('./animation.gif', converted_images, fps=10)
  return embed.embed_file('./animation.gif')
"""

"def to_gif(images):\n  converted_images = np.clip(images * 255, 0, 255).astype(np.uint8)\n  imageio.mimsave('./animation.gif', converted_images, fps=10)\n  return embed.embed_file('./animation.gif')\n"

In [11]:
#to_gif(sample_video)

class FrameGenerator:
  def __init__(self, training = False):
    """ Returns a set of frames with their associated label. 

      Args:
        path: Video file paths.
        n_frames: Number of frames. 
        training: Boolean to determine if training dataset is being created.
    """
    self.training = training
    
    self.class_names = sorted(set(p.name for p in video_filenames.iterdir() if p.is_dir()))
    self.class_ids_for_name = dict((name, idx) for idx, name in enumerate(self.class_names))

  def __call__(self):

    if self.training:
      random.shuffle()

    for path, name in pairs:
      video_frames = frames_from_video_file(path) 
      label = self.class_ids_for_name[name] # Encode labels
      yield video_frames, label

In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox
from PIL import Image, ImageTk

VIDEO_DIR = "../Videos"
CSV_DIR = "../Annotations"
TIMESTEP = 0.5

#Tábla nevek
COLUMNS = ["timestamp", "20","30","50","60","70","80",
               "sebKorlatVege","100","120","elozniTilos",
               "teherautovalElozniTilos","keresztezodesAlarendeltUttal",
               "foutvonal","elsobbsegadas","stop","mind2BehajtaniTilos",
               "teherautovalBehajtaniTilos","behajtaniTilos",
               "veszely","veszelyesKanyarBal","veszelyesKanyarJobb",
               "veszelyesKanyarok","egyenetlenUttsest","csuszos","utszukuletJobbrol",
               "munka","fenyjelzoKeszulek","gyalogosAtkeles","gyerekek",
               "kerekparosok","?","vadVeszely","korlatozasVege",
               "kotelezoHaladasiIranyJobbra","kotelezoHaladasiIranyBalra",
               "kotelezoHaladasiIrany","kotelezoHaladasiIranyJobbraEgyenesen",
               "kotelezoHaladasiIranyBalraEgyenesen","kerulesiIranyJobbra",
               "kerulesiIranyBalra","korforgalom","elozniTilosVege",
               "teherautovalElozniTilosVege","40","autopalya","autopalyaVege",
               "autout","autoutVege","buszmegallo","egyiranyu","foutVege",
               "gyalogosAtkelo","korforgalomPiros","parkolo",
               "szembejovoForgalomnakElsobbseg","szembejovoForgalommalSzembenElsobbseg",
               "utszukulet","utszukuletBalrol","veszelyesKanyarok","zsakutca",
               "gyalogosok","varakozniTilos","megallniTilos","jobbraKanyarodniTilos",
               "balraKanyarodniTilos","behajtaniTilosMegjeloltDolgokkal",
               "behajtaniTilosKerekparral","kerulesiIranyBalVagyJobb",
               "behajtaniTilosSulykorlat","bukkano"
]

class VideoAnnotator:
    def __init__(self, root):
        self.root = root
        self.root.title("Közlekedési tábla annotáló")

        self.video_files = sorted(f for f in os.listdir(VIDEO_DIR) if f.endswith(".mp4"))
        self.current_index = 242
    

        self.canvas = tk.Canvas(root, width=1280, height=720)
        self.canvas.grid(row=0, column=0, columnspan=3)

        self.var_dict = {name: tk.IntVar() for name in COLUMNS}
        self.check_frame = tk.Frame(root)
        self.check_frame.grid(row=1, column=0, columnspan=3)

        for i, name in enumerate(COLUMNS):
            cb = tk.Checkbutton(self.check_frame, text=name, variable=self.var_dict[name])
            cb.grid(row=i//10, column=i%10, sticky='w')

        self.time_label = tk.Label(root, text="Idő: 0.0s")
        self.time_label.grid(row=2, column=0)

        self.prev_btn = tk.Button(root, text="<< Vissza", command=self.prev_frame)
        self.prev_btn.grid(row=2, column=1)

        self.next_btn = tk.Button(root, text=">> Előre", command=self.next_frame)
        self.next_btn.grid(row=2, column=2)

        self.root.protocol("WM_DELETE_WINDOW", self.on_closing)

        self.load_video()

    def load_video(self):
        video_file = self.video_files[self.current_index]
        self.cap = cv2.VideoCapture(os.path.join(VIDEO_DIR, video_file))
        self.fps = self.cap.get(cv2.CAP_PROP_FPS)

        csv_path = os.path.join(CSV_DIR, video_file.replace(".mp4", ".csv"))
        self.df = pd.read_csv(csv_path)

        self.frame_index = 0
        self.update_frame()

    def update_frame(self):
        timestamp = self.df.iloc[self.frame_index]['timestamp']
        self.time_label.config(text=f"Idő: {timestamp:.1f}s")
        frame_number = int(timestamp * self.fps)
        self.cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
        ret, frame = self.cap.read()
        if ret:
            frame = cv2.resize(frame, (1280, 720))
            img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            imgtk = ImageTk.PhotoImage(image=img)
            self.canvas.imgtk = imgtk
            self.canvas.create_image(0, 0, anchor='nw', image=imgtk)

        for name in COLUMNS:
            self.var_dict[name].set(int(self.df.iloc[self.frame_index].get(name, 0)))

    def save_frame(self):
        for name in COLUMNS:
            self.df.at[self.frame_index, name] = self.var_dict[name].get()

    def next_frame(self):
        self.save_frame()
        if self.frame_index + 1 < len(self.df):
            self.frame_index += 1
            self.update_frame()
        else:
            self.save_csv()
            messagebox.showinfo("Kész", "Elértél a videó végére.")

    def prev_frame(self):
        self.save_frame()
        if self.frame_index > 0:
            self.frame_index -= 1
            self.update_frame()

    def save_csv(self):
        video_file = self.video_files[self.current_index]
        csv_path = os.path.join(CSV_DIR, video_file.replace(".mp4", ".csv"))
        self.df.to_csv(csv_path, index=False)

    def on_closing(self):
        self.save_frame()
        self.save_csv()
        self.root.destroy()

if __name__ == '__main__':
    root = tk.Tk()
    app = VideoAnnotator(root)
    root.mainloop()

In [ ]:

"""VIDEO_DIR = r"D:\Szakdolgozat\kozlekedesi_tabla_felismero\Videos"
OUTPUT_DIR = r"D:\Szakdolgozat\kozlekedesi_tabla_felismero\ExtractedFrames"
TIMESTEP = 0.5  # másodperc
FRAMES_PER_STEP = 5  # ennyi frame minden 0.5 mp-n belül

os.makedirs(OUTPUT_DIR, exist_ok=True)

video_files = [f for f in os.listdir(VIDEO_DIR) if f.endswith(".mp4")]

for video_file in video_files:
    video_path = os.path.join(VIDEO_DIR, video_file)
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration_sec = total_frames / fps

    print(f"Feldolgozás: {video_file} ({duration_sec:.1f} mp)")

    num_steps = int(duration_sec / TIMESTEP)

    # csak azokat a sorokat tartjuk meg, ahol van legalább egy 1 az oszlopok között
    df = df[df.drop(columns=['timestamp']).sum(axis=1) > 0]


    for step in range(num_steps):
        timestamp = step * TIMESTEP
        center_frame = int(timestamp * fps)
        offsets = np.linspace(-2, 2, FRAMES_PER_STEP).astype(int)  # pl. [-2, -1, 0, 1, 2]

        for i, offset in enumerate(offsets):
            frame_idx = center_frame + offset
            if frame_idx < 0 or frame_idx >= total_frames:
                continue
            cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
            ret, frame = cap.read()
            if ret:
                resized = cv2.resize(frame, (640, 360))
                output_name = f"{os.path.splitext(video_file)[0]}_{timestamp:.1f}_{i+1}.png"
                output_path = os.path.join(OUTPUT_DIR, output_name)
                cv2.imwrite(output_path, resized)

    cap.release()

print("✅ Minden videó feldolgozva.")


VIDEO_DIR = r"D:\Szakdolgozat\kozlekedesi_tabla_felismero\Videos"
CSV_DIR = r"D:\Szakdolgozat\kozlekedesi_tabla_felismero\Annotations"
OUTPUT_DIR = r"D:\Szakdolgozat\kozlekedesi_tabla_felismero\ExtractedFrames"

os.makedirs(OUTPUT_DIR, exist_ok=True)

for filename in os.listdir(VIDEO_DIR):
    if filename.endswith(".mp4"):
        video_path = os.path.join(VIDEO_DIR, filename)
        csv_path = os.path.join(CSV_DIR, filename.replace(".mp4", ".csv"))
        df = pd.read_csv(csv_path)

        # csak azok a sorok kellenek, ahol legalább egy oszlopban 1 van (kivéve a timestamp)
        df_filtered = df[df.drop(columns=['timestamp']).sum(axis=1) > 0]

        cap = cv2.VideoCapture(video_path)
        fps = cap.get(cv2.CAP_PROP_FPS)

        for idx, row in df_filtered.iterrows():
            timestamp = row['timestamp']
            frame_number = int(timestamp * fps)
            cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
            ret, frame = cap.read()
            if ret:
                out_name = f"{filename[:-4]}_{timestamp:.1f}.jpg"
                out_path = os.path.join(OUTPUT_DIR, out_name)
                cv2.imwrite(out_path, frame)

        cap.release()
"""

import os
import cv2
import pandas as pd
from PIL import Image

# --- Beállítások ---
VIDEO_DIR = "../Videos"
CSV_DIR = "../Annotations"
OUTPUT_DIR = "../ExtractedFrames"
IMAGE_SIZE = (512, 512)
FRAMES_PER_TIMESTAMP = 3  # Képkockák száma fél másodpercenként

os.makedirs(OUTPUT_DIR, exist_ok=True)

# Osztályok listája (timestamp nélkül)
with open(os.path.join(CSV_DIR, os.listdir(CSV_DIR)[0]), 'r') as f:
    header = f.readline().strip().split(',')

COLUMNS = [col for col in header if col != 'timestamp']

for video_file in sorted(os.listdir(VIDEO_DIR)):
    if not video_file.endswith(".mp4"):
        continue

    base_name = os.path.splitext(video_file)[0]
    video_path = os.path.join(VIDEO_DIR, video_file)
    csv_path = os.path.join(CSV_DIR, f"{base_name}.csv")

    df = pd.read_csv(csv_path)
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)

    for _, row in df.iterrows():
        timestamp = row['timestamp']
        active_labels = [label for label in COLUMNS if row.get(label, 0) == 1]
        if not active_labels:
            continue

        # Időbélyeg képkockaszámra
        center_frame = int(timestamp * fps)
        offsets = [-1, 0, 1]  # 3 frame: középső + 1 előtte + 1 utána

        for offset in offsets:
            frame_num = center_frame + offset
            cap.set(cv2.CAP_PROP_POS_FRAMES, frame_num)
            ret, frame = cap.read()
            if not ret:
                continue

            resized = cv2.resize(frame, IMAGE_SIZE)

            for label in active_labels:
                label_dir = os.path.join(OUTPUT_DIR, label)
                os.makedirs(label_dir, exist_ok=True)
                filename = f"{base_name}_{timestamp:.1f}_{frame_num}.jpg"
                path = os.path.join(label_dir, filename)
                cv2.imwrite(path, resized)

    cap.release()

print("Képkockák mentése kész.")




Képkockák mentése kész.


In [ ]:

from tensorflow.keras.utils import image_dataset_from_directory
import matplotlib.pyplot as plt
import os



BATCH_SIZE = 32
IMG_SIZE = (512, 512)

dataset_dir = "../ExtractedFrames"

train_ds = image_dataset_from_directory(
    dataset_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

val_ds = image_dataset_from_directory(
    dataset_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

# Normalizálás 0-1 közé
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))


Found 11942 files belonging to 49 classes.
Using 9554 files for training.
Found 11942 files belonging to 49 classes.
Using 2388 files for validation.


In [13]:
fg = FrameGenerator(training=True)

frames, label = next(fg())

print(f"Shape: {frames.shape}")
print(f"Label: {label}")

NameError: name 'FrameGenerator' is not defined

In [ ]:
# Create the training set
output_signature = (tf.TensorSpec(shape = (None, None, None, 3), dtype = tf.float32),
                    tf.TensorSpec(shape = (), dtype = tf.int16))

train_ds = tf.data.Dataset.from_generator(FrameGenerator(#  10, 
                                                         training=True),
                                          output_signature = output_signature)

In [ ]:
# Create the validation set
val_ds = tf.data.Dataset.from_generator(FrameGenerator(training=False),
                                        output_signature = output_signature)

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size = AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size = AUTOTUNE)

In [ ]:
# Print the shapes of the data
train_frames, train_labels = next(iter(train_ds))
print(f'Shape of training set of frames: {train_frames.shape}')
print(f'Shape of training labels: {train_labels.shape}')

val_frames, val_labels = next(iter(val_ds))
print(f'Shape of validation set of frames: {val_frames.shape}')
print(f'Shape of validation labels: {val_labels.shape}')

In [ ]:
train_ds = train_ds.batch(2)
val_ds = val_ds.batch(2)

train_frames, train_labels = next(iter(train_ds))
print(f'Shape of training set of frames: {train_frames.shape}')
print(f'Shape of training labels: {train_labels.shape}')

val_frames, val_labels = next(iter(val_ds))
print(f'Shape of validation set of frames: {val_frames.shape}')
print(f'Shape of validation labels: {val_labels.shape}')

In [ ]:
# Átalakítás numpy tömbökké
train_images = np.array(train_images) / 255.0  # Normalizálás
test_images = np.array(test_images) / 255.0

train_labels = to_categorical(train_labels, num_classes)  # One-hot encoding
test_labels = to_categorical(test_labels, num_classes)

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, make_scorer, mean_squared_log_error, \
    root_mean_squared_error

# Neural Network with TensorFlow

device = "GPU" if tf.config.list_physical_devices('GPU') else "CPU"
print(f"Using device: {device}")

# Hyperparameters
batch_size = 256

activation_func = 'relu'

def scheduler(epoch, lr):
    if epoch < 550:
        return lr
    else:
        return lr * np.exp(-0.1)

num_epochs = 50
learning_rate = 1.5e-4
criterion = tf.keras.losses.MeanSquaredError()
# criterion = tf.keras.losses.MeanAbsoluteError()

optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, epsilon=1e-5)

train_dataset = tf.data.Dataset.from_tensor_slices((tf.convert_to_tensor(train_images, dtype=tf.float32),
                                                    tf.convert_to_tensor(train_labels, dtype=tf.float32)))
val_dataset = tf.data.Dataset.from_tensor_slices((tf.convert_to_tensor(test_images, dtype=tf.float32),
                                                    tf.convert_to_tensor(test_labels, dtype=tf.float32)))

train_loader = train_dataset.batch(batch_size, drop_remainder=True) # .shuffle(buffer_size=len(train_images))
val_loader = val_dataset.batch(batch_size, drop_remainder=True) # .shuffle(buffer_size=len(test_images))



Using device: CPU


In [ ]:

class Net(tf.keras.Model):
    def __init__(self):
        super(Net, self).__init__()

        self.input_layer = tf.keras.layers.Dense((IMG_SIZE, IMG_SIZE, batch_size), activation=activation_func)

        # self.feature_extractor = []
        # for i in range(5):
        #     self.hidden_layers.append(tf.keras.layers.Dense(2048, activation=activation_func))
        #     self.hidden_layers.append(tf.keras.layers.Dropout(0.2))

        self.hidden_layers = []
        for i in range(7):
            self.hidden_layers.append(tf.keras.layers.Dense(512, activation=activation_func))
            self.hidden_layers.append(tf.keras.layers.Dropout(0.2))
            self.hidden_layers.append(tf.keras.layers.BatchNormalization())

        # self.additional_layers = []
        # for i in range(2):
        #     self.additional_layers.append(tf.keras.layers.Dense(256, activation=activation_func))
        #     self.additional_layers.append(tf.keras.layers.Dropout(0.2))

        self.output_layer = tf.keras.layers.Dense(num_classes, activation='softmax')

    def call(self, x):
        x = self.input_layer(x)
        # for layer in self.feature_extractor:
        #     x = layer(x)
        for layer in self.hidden_layers:
            x = layer(x)
        # for layer in self.additional_layers:
        #     x = layer(x)

        return self.output_layer(x)
    

model = Net()

In [ ]:


train_losses = []
val_losses = []
val_accuracies = []

for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")

    train_loss = 0
    for step, (batch_x, batch_y) in enumerate(train_loader):
        with tf.GradientTape() as tape:
            outputs = model(batch_x, training=True)
            loss = criterion(batch_y, tf.squeeze(outputs))

        grads = tape.gradient(loss, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        train_loss += loss.numpy()

    train_losses.append(train_loss / len(train_loader))

    # Validation loop
    val_loss = 0
    for batch_x, batch_y in val_loader:
        outputs = model(batch_x, training=False)
        loss = criterion(batch_y, tf.squeeze(outputs))
        val_loss += loss.numpy()

    val_losses.append(val_loss / len(val_loader))

    print(
        f"Epoch {epoch + 1}/{num_epochs}, "
        f"Train Loss: {train_loss / len(train_loader):.4f}, "
        f"Validation Loss: {val_loss / len(val_loader):.4f}"
    )

train_losses = np.array(train_losses)
val_losses = np.array(val_losses)

# Making predictions
x_train_tensor = tf.convert_to_tensor(train_dataset, dtype=tf.float32)
x_test_tensor = tf.convert_to_tensor(val_dataset, dtype=tf.float32)

y_train_pred = model(x_train_tensor, training=False).numpy()
y_test_pred = model(x_test_tensor, training=False).numpy()


Epoch 1/50


ValueError: Exception encountered when calling Net.call().

[1mInvalid dtype: tuple[0m

Arguments received by Net.call():
  • x=tf.Tensor(shape=(256, 32, 32, 3), dtype=float32)

In [ ]:

train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)
test_mse = mean_squared_error(y_test, y_test_pred)
test_rmse = np.sqrt(test_mse)
test_mae = mean_absolute_error(y_test, y_test_pred)

print(f'Train R-squared: {train_r2 * 100:.2f}%')
print(f'Test R-squared: {test_r2 * 100:.2f}%')
print(f'Test Mean Squared Error (MSE): {test_mse:.2f}')
print(f'Test Root Mean Squared Error (RMSE): {test_rmse:.2f}')
print(f'Test Mean Absolute Error (MAE): {test_mae:.2f}')

plt.figure(figsize=(10, 5))
plt.plot(y_test, label='Actual', marker='o', linestyle='None')
plt.plot(y_test_pred, label='Predicted', marker='x', linestyle='None')
plt.legend()
plt.xlabel('Sample index')
plt.ylabel('Value')
plt.title('Actual vs Predicted Values')
plt.show()

plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.show()

print(model.summary())
print(model.get_config())
print(optimizer.get_config())

model.save("HousePrices/saved_models/nn_model_" + exp_name + ".tf",
            save_format='tf')

NameError: name 'r2_score' is not defined